#### 두개의 투자전략을 클래스로 선언
1. 생성자 함수 
    - 클래스가 생성될 때 최초로 한번만 실행이 되는 함수 
    - 객체 변수(self 변수)에 데이터프레임, 기준이 되는 컬럼명, 시작시간, 종료시간
2. buyandhold 함수 
    - 생성자 함수에서 만든 객체 변수를 이용하여 백태스팅
    - 결과와 누적 수익율을 리턴 
3. bollinger 함수
    - 밴드 생성 함수 
        - 상단 밴드, 하단 밴드, 이동 평균선 생성
    - 거래 내역 추가 함수
        - 밴드를 기준으로 거래 내역을 생성
    - 수익율 계산 함수
        - 매도 시 수익율 발생하고 데이터프레임과 누적 수익율을 리턴


In [61]:
from datetime import datetime
import pandas as pd
import numpy as np

In [62]:
class Invest:
    # 생성자 함수 
    def __init__(self, 
                 _df, 
                 _col = 'Adj Close', 
                 _start = "2010-01-01", 
                 _end = datetime.now()):
        # _df의 결측치, 무한대를 제외시킨다. 
        flag = _df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
        self.df = _df.loc[~flag,]
        # 데이터프레임에서 컬럼에 Date가 포함되어있는가?
        if 'Date' in self.df.columns:
            self.df.set_index('Date', inplace=True)
        # 인덱스를 시계열데이터로 변경
        self.df.index = pd.to_datetime(self.df.index, format='%Y-%m-%d')
        self.df = self.df[[_col]]
        self.col = _col
        try:
            self.start = datetime.strptime(_start, '%Y-%m-%d')
            if type(_end) == 'str':
                self.end = datetime.strptime(_end, '%Y-%m-%d')
            else:
                self.end = _end
        except:
            print('투자기간의 인자값의 포멧이 잘못되었습니다.(YYYY-mm-dd)')
    ## buyandhold 함수를 생성
    def buyandhold(self):
        # 투자기간으로 데이터를 필터링 
        result = self.df.loc[self.start : self.end, ]
        # 일일 수익율 컬럼을 생성
        result['rtn'] = (result[self.col].pct_change() + 1).fillna(1)
        # 누적 수익율 컬럼을 생성 
        result['acc_rtn'] = result['rtn'].cumprod()
        # 총 누적 수익율 데이터만 추출
        acc_rtn = result.iloc[-1,]['acc_rtn']
        return result, acc_rtn
    # bollinger 함수
    # 밴드 생성 함수 생성
    def bollinger(self, _cnt = 20):
        result = self.df.copy()
        # 이동 평균선 생성
        result['center'] = result[self.col].rolling(_cnt).mean()
        # 상단 밴드를 생성
        result['ub'] = result['center'] + (2 * result[self.col].rolling(_cnt).std())
        # 하단 밴드를 생성 
        result['lb'] = result['center'] - (2 * result[self.col].rolling(_cnt).std())
        # 투자기간으로 데이터를 필터 
        result = result.loc[self.start : self.end, ]
        # 거래내역 추가
        result['trade'] = ""
        for i in result.index:
            # 상단 밴드보다 기준의 되는 컬럼의 데이터가 크거나 같은 경우 
            if result.loc[i, self.col] >= result.loc[i, 'ub']:
                # 현재 보유중이라면 매도, 보유중이 아니라면 유지 
                result.loc[i, 'trade'] = ""
            # 하단 밴드보다 기준이 되는 컬럼의 데이터가 작거나 같은 경우
            elif result.loc[i, self.col] <= result.loc[i, 'lb']:
                # 현재 보유중이 아니라면 매수, 보유 중이라면 유지
                result.loc[i, 'trade'] = 'buy'
            # 밴드 사이에 기준되는 컬럼의 데이터가 존재한다면
            else:
                # 현재 보유중이라면 보유, 보유 중이 아니라면 보유하지 않음
                if (result.shift().loc[i, 'trade'] == ""):
                    result.loc[i, 'trade'] = result.shift().loc[i, 'trade']

        # 수익율 계산
        result['rtn'] = 1

        for i in result.index:
            # 매수의 경우 ( 전날의 trade가 "" 오늘의 trade가 "buy"인 경우 )
            if (result.shift().loc[i, 'trade'] == "") & (result.loc[i, 'trade'] == "buy"):
                # 구매가 대입 
                buy = result.loc[i, self.col]
                print(f"매수일 : {i}, 매수가 : {buy}")
            # 매도의 경우 ( 전날의 trade가 "buy" 오늘의 trade가 ""인 경우 )
            elif (result.shift().loc[i, 'trade'] == "buy") & (result.loc[i, 'trade'] == ""):
                # 판매가 대입
                sell = result.loc[i, self.col]
                # 수익율 계산
                rtn = sell / buy
                # 수익율을 판매한 날의 rtn 컬럼에 대입 
                result.loc[i, 'rtn'] = rtn
                print(f"매도일 : {i}, 매도가 : {sell}, 수익율 : {round(rtn, 2)}")
        # 누적 수익율 계산
        result['acc_rtn'] = result['rtn'].cumprod()
        # 최종 누적 수익율을 변수에 대입 
        acc_rtn = result.iloc[-1,]['acc_rtn']


        return result, acc_rtn



In [63]:
AAPL = pd.read_csv('../../csv/AAPl.csv')

In [64]:
class1 = Invest(AAPL)

In [65]:
bnh_df, bnh_rtn = class1.buyandhold()

C:\Users\moons\AppData\Local\Temp\ipykernel_668\3890840902.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['rtn'] = (result[self.col].pct_change() + 1).fillna(1)
C:\Users\moons\AppData\Local\Temp\ipykernel_668\3890840902.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['acc_rtn'] = result['rtn'].cumprod()


In [66]:
bnh_rtn

7.436513727083075

In [67]:
bnh_df.tail()

,Adj Close,rtn,acc_rtn
Date,,,
2019-06-18,198.449997,1.023518,7.409631
2019-06-19,197.869995,0.997077,7.387975
2019-06-20,199.460007,1.008036,7.447342
2019-06-21,198.779999,0.996591,7.421952
2019-06-24,199.169998,1.001962,7.436514


In [68]:
df, rtn =class1.bollinger()

매수일 : 2010-01-22 00:00:00, 매수가 : 24.747818
매도일 : 2010-01-25 00:00:00, 매도가 : 25.413599, 수익율 : 1.03
매수일 : 2010-01-28 00:00:00, 매수가 : 24.940546
매도일 : 2010-02-01 00:00:00, 매도가 : 24.369871, 수익율 : 0.98
매수일 : 2010-08-24 00:00:00, 매수가 : 30.026524
매도일 : 2010-08-25 00:00:00, 매도가 : 30.396961, 수익율 : 1.01
매수일 : 2011-03-16 00:00:00, 매수가 : 41.299767
매도일 : 2011-03-17 00:00:00, 매도가 : 41.879189, 수익율 : 1.01
매수일 : 2011-05-16 00:00:00, 매수가 : 41.711502
매도일 : 2011-05-17 00:00:00, 매도가 : 42.066914, 수익율 : 1.01
매수일 : 2011-06-10 00:00:00, 매수가 : 40.785408
매도일 : 2011-06-13 00:00:00, 매도가 : 40.873005, 수익율 : 1.0
매수일 : 2011-06-20 00:00:00, 매수가 : 39.461357
매도일 : 2011-06-21 00:00:00, 매도가 : 40.710316, 수익율 : 1.03
매수일 : 2011-11-14 00:00:00, 매수가 : 47.463268
매도일 : 2011-11-15 00:00:00, 매도가 : 48.660904, 수익율 : 1.03
매수일 : 2011-11-21 00:00:00, 매수가 : 46.180508
매도일 : 2011-11-22 00:00:00, 매도가 : 47.119102, 수익율 : 1.02
매수일 : 2012-04-16 00:00:00, 매수가 : 72.601524
매도일 : 2012-04-17 00:00:00, 매도가 : 76.302109, 수익율 : 1.05
매수일 : 2012-04-20 00:0

C:\Users\moons\AppData\Local\Temp\ipykernel_668\3890840902.py:82: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1.0269026142021895' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.loc[i, 'rtn'] = rtn


매수일 : 2012-10-08 00:00:00, 매수가 : 80.207954
매도일 : 2012-10-09 00:00:00, 매도가 : 79.916374, 수익율 : 1.0
매수일 : 2012-10-19 00:00:00, 매수가 : 76.647339
매도일 : 2012-10-22 00:00:00, 매도가 : 79.68763, 수익율 : 1.04
매수일 : 2012-11-02 00:00:00, 매수가 : 72.494728
매도일 : 2012-11-05 00:00:00, 매도가 : 73.477585, 수익율 : 1.01
매수일 : 2012-11-07 00:00:00, 매수가 : 70.452194
매도일 : 2012-11-09 00:00:00, 매도가 : 69.0709, 수익율 : 0.98
매수일 : 2013-01-15 00:00:00, 매수가 : 61.351456
매도일 : 2013-01-16 00:00:00, 매도가 : 63.898102, 수익율 : 1.04
매수일 : 2013-01-24 00:00:00, 매수가 : 56.879379
매도일 : 2013-01-28 00:00:00, 매도가 : 56.7948, 수익율 : 1.0
매수일 : 2013-03-04 00:00:00, 매수가 : 53.343914
매도일 : 2013-03-05 00:00:00, 매도가 : 54.752277, 수익율 : 1.03
매수일 : 2013-04-17 00:00:00, 매수가 : 51.153259
매도일 : 2013-04-19 00:00:00, 매도가 : 49.595051, 수익율 : 0.97
매수일 : 2013-06-19 00:00:00, 매수가 : 54.074104
매도일 : 2013-06-26 00:00:00, 매도가 : 50.887192, 수익율 : 0.94
매수일 : 2013-09-11 00:00:00, 매수가 : 60.184383
매도일 : 2013-09-17 00:00:00, 매도가 : 58.590046, 수익율 : 0.97
매수일 : 2014-01-03 00:00:00, 

In [69]:
flag = (df['trade'] != 'buy')
df.loc[flag,]

,Adj Close,center,ub,lb,trade,rtn,acc_rtn
Date,,,,,,,
2010-01-04,26.782711,25.037723,27.046734,23.028713,,1.0,1.00000
2010-01-05,26.829010,25.169503,27.288098,23.050908,,1.0,1.00000
2010-01-06,26.402260,25.307290,27.366449,23.248130,,1.0,1.00000
2010-01-07,26.353460,25.436879,27.410937,23.462821,,1.0,1.00000
2010-01-08,26.528664,25.525609,27.529742,23.521475,,1.0,1.00000
...,...,...,...,...,...,...,...
2019-06-18,198.449997,185.432500,201.032574,169.832427,,1.0,1.42265
2019-06-19,197.869995,185.996000,202.558154,169.433846,,1.0,1.42265
2019-06-20,199.460007,186.830000,204.361771,169.298229,,1.0,1.42265
